In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import statsmodels.api as sm
import numpy as np

In [2]:
factors=pdr.DataReader('F-F_Research_Data_Factors_daily','famafrench',start='1990-1-1')[0]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20836\3697599442.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors=pdr.DataReader('F-F_Research_Data_Factors_daily','famafrench',start='1990-1-1')[0]


In [3]:
import yfinance as yf
yf.pdr_override()

tickers=['AAPL']
sec_data=pd.DataFrame()
for t in tickers:
	sec_data[t]=pdr.get_data_yahoo(t,start='1990-1-1',end='2024-3-28')['Adj Close']

sec_returns=sec_data.pct_change().dropna()*100

all=pd.merge(sec_returns,factors,left_index=True,right_index=True).dropna()
all.head()

[*********************100%%**********************]  1 of 1 completed


,AAPL,Mkt-RF,SMB,HML,RF
Date,,,,,
1990-01-03,0.671104,-0.06,0.73,-0.26,0.026
1990-01-04,0.333648,-0.71,0.42,-0.22,0.026
1990-01-05,0.332190,-0.85,0.77,-0.21,0.026
1990-01-08,0.662203,0.30,-0.41,-0.26,0.026
1990-01-09,-0.986758,-1.01,0.87,0.07,0.026


In [7]:
all['AAPL-RF'] = all['AAPL'] - all['RF']
X = all[['Mkt-RF']]
X1 = sm.add_constant(X)
reg = sm.OLS(all['AAPL-RF'], X1).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                AAPL-RF   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     2817.
Date:                Sat, 18 May 2024   Prob (F-statistic):               0.00
Time:                        22:08:16   Log-Likelihood:                -19573.
No. Observations:                8624   AIC:                         3.915e+04
Df Residuals:                    8622   BIC:                         3.916e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0603      0.025      2.391      0.017       0.011       0.110
Mkt-RF         1.1689      0.022     53.079      0.000       1.126       1.212
==============================================================================
Omnibus:                     2887.278   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           499466.307
Skew:                          -0.435   Prob(JB):                         0.00
Kurtosis:                      40.272   Cond. No.                         1.15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""